# Shopeeコンペ概略&#x1f600;
1.  インスタなどをイメージしてもらえるとイメージしやすいと思うのですが、データには画像とハッシュタグ、タイトルとそれぞれの投稿に対応するposting idがあります。
2.  テストデータの画像群とそれぞれの画像のハッシュタグ、タイトルなどの情報から、テストデータ内で、同じ製品をグループ分けしてそれを記載して提出するコンペだと思います。
3.  トレインデータもテストデータ同様、画像、ハッシュタブ、タイトルなどがありますが、テストデータの項目にはないラベルイメージグループという項目があって、同じ製品でグループ分けしてくれています。その情報を使ってやるものと推測します。

## 参考までに、本codeは、
### https://www.kaggle.com/chumajin/eda-of-shopee-for-starter
### を和訳と解説を加えたもので, 周囲への説明用ですが
### もし、参考になりましたら、upvoteお願いいたします! 参考元の英語版もしてくれると、なお嬉しいです。


In [ ]:
import numpy as np 
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt

# 0. csvファイルの確認

In [ ]:
traindf = pd.read_csv("../input/shopee-product-matching/train.csv")
traindf

In [ ]:
testdf = pd.read_csv("../input/shopee-product-matching/test.csv")
testdf

In [ ]:
sample = pd.read_csv("../input/shopee-product-matching/sample_submission.csv")
sample

In [ ]:
TRAIN_PATH = "../input/shopee-product-matching/train_images"
TEST_PATH = "../input/shopee-product-matching/test_images"

traindf["path"] = [os.path.join(TRAIN_PATH,s) for s in traindf["image"]]
testdf["path"] = [os.path.join(TEST_PATH,s) for s in testdf["image"]]

traindf.to_csv("traindf.csv",index=False)
testdf.to_csv("testdf.csv",index=False)

# 1. Train.csvの確認

In [ ]:
traindf.head(3)

In [ ]:
traindf.info()

* nanデータはなし。全部で34250行ある

In [ ]:
for col in traindf.columns:
    print(col + ":" + str(len(traindf[col].unique())))

* この結果は、各カラムにどれだけユニークなワードがあるかを示しています。
* トータルで34250。posting id以外は重複箇所がある。
* コンペの性質上、label_groupに重複があっても良いと思いますが、他にもimage, image_phash, titleにも重複があります。

## 1.1 とりあえず絵を表示してみます。

In [ ]:
img = cv2.imread(traindf["path"].iloc[0])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
tmpdf = traindf[traindf["label_group"]==traindf["label_group"].iloc[0]]
tmpdf

In [ ]:
for a in tmpdf["path"]:
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure()
    plt.imshow(img)
    plt.axis("off")

* 両方Victoria secretのもので、同じラベルグループに同じ製品があることが確認できました。

## 1.2 同じlabel_groupsについていろいろ解析してみました。

### 1.2.1 それぞれのlabel_groupに何枚の絵があるかをまず数えました。

In [ ]:
labels = traindf.groupby("label_group")["image"].count().reset_index()
labels.columns=["label_group","image_num"]
labels

In [ ]:
sortlabels = labels.sort_values("image_num")
sortlabels

** 最小枚数は2枚。最大枚数は51枚であることが確認されました。

### 1.2.2 枚数ごとにlabel_groupが何個あるかを把握。例えば、2枚しかないlabel_groupは何個存在するのか?とかそういう意味です。

In [ ]:
imagecount = labels.groupby("image_num").count().reset_index()
imagecount.columns=["image_num","counts"]
imagecount

* ほとんどが2枚しかないことが確認されます。

In [ ]:
plt.bar(imagecount["image_num"],imagecount["counts"])

### 1.2.3 label_groupの画像数が最も多いグループの画像を表示してみます。 

In [ ]:
tmpdf = traindf[traindf["label_group"]==sortlabels["label_group"].iloc[-1]]
tmpdf.head(5)
    

In [ ]:
plt.figure(figsize=(20,20))

for num,a in enumerate(tmpdf["path"]):
    plt.subplot(11,5,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.imshow(img)
    
    

## 1.3 train.csvの中でも同じimage(画像)の解析

In [ ]:
imagegroup = traindf.groupby("image")["path"].count().reset_index()
imagegroup.columns=["image","counts"]
imagegroup

In [ ]:
tmpdf = imagegroup[imagegroup["counts"] > 1]
tmpdf.sort_values("counts")
tmpdf

* 同じイメージを使用しているものは多くても2枚ということが確認されました。

In [ ]:
tmpdf["image"].iloc[0]

In [ ]:
traindf[traindf["image"]==tmpdf["image"].iloc[0]]

* posting idだけ変わっていることが確認されました。この場合、他は同じです。

In [ ]:
traindf[traindf["image"]==tmpdf["image"].iloc[1]]

* 他の例を見ると、タイトルだけ異なっているので、編集とかしたのかなと推測されます。

In [ ]:
traindf.groupby(["image","image_phash"])["path"].count().reset_index()

* imageとimage_phashのグループは、32412とimageの数と同じなので、少なくとも、imageが同じときは、image_phashも同じであることが確認されました。

## 1.4 train.csvの中の同じimage_phashの解析

In [ ]:
phashgroup = traindf.groupby("image_phash")["path"].count().reset_index()
phashgroup.columns=["image_phash","counts"]
phashgroup

In [ ]:
sortphash = phashgroup.sort_values("counts")
sortphash

* 同じハッシュタグは、26個が最大

In [ ]:
tmpdf = traindf[traindf["image_phash"]==sortphash["image_phash"].iloc[-1]]
tmpdf

* 同じハッシュなのだが、imageが異なっています。最も多かった画像を見てみます。

In [ ]:
plt.figure(figsize=(20,20))

for num,a in enumerate(tmpdf["path"]):
    plt.subplot(6,5,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.axis("off")
    plt.title(tmpdf["posting_id"].iloc[num])
    plt.imshow(img)
    

* 画像はことなりますが、全部同じに見えます。

## 1.5 train.csvの中の同じtitlesの解析

In [ ]:
traindf

In [ ]:
titlegroup = traindf.groupby("title")["path"].count().reset_index()
titlegroup.columns=["title","counts"]
titlegroup

In [ ]:
sorttitle = titlegroup.sort_values("counts")
sorttitle

* 同じタイトルは最大で9個。最大となって画像を表示してみます。

In [ ]:
tmpdf = traindf[traindf["title"]==sorttitle["title"].iloc[-1]]
tmpdf

In [ ]:
plt.figure(figsize=(20,20))

for num,a in enumerate(tmpdf["path"]):
    plt.subplot(3,5,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.axis("off")
    plt.imshow(img)
    

* タイトルは同じだけど、画像は異なる。ラベルグループも同じだったり、違ったりです。

# 2. test.csvについて

In [ ]:
testdf

In [ ]:
plt.figure(figsize=(20,20))

for num,a in enumerate(testdf["path"]):
    
    plt.subplot(1,3,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(img)
    plt.axis("off")
    

* ここでは3枚しかありません。

# 3.submission.csvについて

In [ ]:
sample

## submission rules

sample_submission.csv - a sample submission file in the correct format.

posting_id - the ID code for the posting.

matches - Space delimited list of all posting IDs that match this posting. Posts always self-match. Group sizes were capped at 50, so there's no need to predict more than 50 matches.


## ↑は、コンペに記載されたものをコピペしたものです。
### matchesのところを見ると、同じ製品ごとに、spaceで区切ってくださいというのと、post idは常に、自分とマッチする。あと、グループサイズのmaxは50で、それ以上は予想する必要がないと記載があります。

# 4.submissionの簡単なテスト
### これまでの結果から、image_phashが同じなら同じグループなのでは?という仮説ができました。
### それをやっても良いのですが、test.csvは3行しかないので、書式が本当にあっているのか不安です。そのため、train.csvで簡易的にテストしてみました。(ちなみに、私は、defaultdictを使いました)


In [ ]:
practicedf = traindf.iloc[:1000,:]

In [ ]:
practicedf

In [ ]:
from collections import defaultdict

In [ ]:
every_phash = defaultdict(list)

In [ ]:
for num, row in enumerate(practicedf[['posting_id', 'image_phash']].values):
    every_phash[row[1]].append(row[0])

In [ ]:
#every_phash

In [ ]:
every_phash_list = []

for num, row in enumerate(practicedf[['posting_id','image_phash']].values):
    pred = ""
    for a in every_phash[row[1]]:
        pred = pred + a + " "

    
    pred=pred[:-1] # delete last space
    
    every_phash_list.append(pred)

In [ ]:
every_phash_list[:20]

### 上から12行目みたいなやつが同じグループで複数個あるケースです。

In [ ]:
practicedf["matches"] = every_phash_list
practicedf

## テストデータの場合。

In [ ]:
testdf

In [ ]:
every_phash = defaultdict(list)

In [ ]:
for num, row in enumerate(testdf[['posting_id', 'image_phash']].values):
    every_phash[row[1]].append(row[0])

In [ ]:
every_phash

In [ ]:
every_phash_list = []

for num, row in enumerate(testdf[['posting_id','image_phash']].values):
    pred = ""
    for a in every_phash[row[1]]:
        pred = pred + a + " "

    
    pred=pred[:-1] # delete last space
    
    every_phash_list.append(pred)

In [ ]:
testdf["matches"] = every_phash_list
testdf

In [ ]:
submission = testdf[["posting_id","matches"]]
submission

In [ ]:
submission.to_csv("submission.csv",index=False)

## インターネットオフにしてsubmitすると、0.559くらいスコアがいきます。
## 最後までお読み頂いてありがとうございます。